# Twitter API to stream Tweets

In diesem Notebook wird der Einsatz der Twitter API für das Streaming von Tweets anhand bestimmter Filterwörter erläutert.  
Für die Twitter API gibt es ein Python Paket mit dem Namen [Tweepy](https://www.tweepy.org/), das die Funktionen der Twitter API für einen kompakteren Zugriff in der Programmiersprache Python ermöglicht.  
Voraussetzung für die Nutzung der API ist ein Twitter Developer Account, mit dem der Zugriff auf die API mittels Authentifizierungstoken geschehen kann.  
Um Tweets streamen zu können, muss eine Klasse erstellt werden, die von der [tweepy.Stream](https://docs.tweepy.org/en/v4.4.0/streaming.html) Klasse erbt und dadurch eine Methode onData() erben kann. Der onData()-Methode wird der empfangene Tweet als Parameter übergeben und kann dann entsprechend verarbeitet werden, in unserem Fall in der MongoDB gespeichert werden.  
Im folgenden wird Schrittweise erläutert, wie wir vorgegangen sind.

Zunächst laden wir die benötigten Pakete für Python.

In [1]:
import os
import json
import pymongo
import pandas as pd 
import tweepy
from dotenv import load_dotenv

### Laden der .env-Datei, in der die Authentifizierungstoken gespeichert sind

In [2]:
load_dotenv('../../.env') # .env file in 'code' dir

True

### Laden der Datei mit den Bezirken und Platznamen

In der CSV-Datei stehen alle Bezirke und Platznamen, die wir als Filterwörter unserem Stream übergeben, die Bezirke und Plätze wurden aus dem [OpenData Portal](https://transparenz.hamburg.de/) der Stadt Hamburg heruntergeladen. 

In [3]:
to_track = pd.read_csv('names_to_track.csv')['0'].tolist()

### Aufsetzen der MongoDB

Als nächstes wird eine Instanz der MongoDB Datenbank definiert. In unserem Falle heißt die Datenbank 'Webgefluester' und die Collection, in der wir alle Tweets ablegen möchten heißt 'streaming_tweets'. Dabei werden entsprechende Instanzen der Datenbank und der Collection erzeugt, wenn sie noch nicht vorhanden sein sollten. 

In [4]:
# Set up MongoDB
client = pymongo.MongoClient('localhost:27017')
db = client["webgefluester"]
collection = db['streaming_tweets']

### Erben von der tweepy.Stream Klasse

Um die Streaming API nutzen zu können und eintreffende Tweets analysieren und speichern zu können, wird von der tweepy.Stream Klasse geerbt und die Methode onData() implementiert. Wenn ein Tweet auf Twitter dem Filter entspricht, wird die Methode onData() aufgerufen und der Tweet übergeben.  
Die Methode lädt den Tweet dann im JSON-Format und fügt einen neuen Schlüssel mit dem Namen '_id' hinzu, der dem Wert des Schlüssels 'id' entspricht. Dadurch speichert die MongoDB die Tweets unter derselben ID ab, wie sie bei Twitter abgespeichert werden und es werden keine Tweets doppelt abgespeichert.  
Als nächstes wird der Tweet in die zuvor definierte Collection eingefügt.

In [5]:
# inherit from tweepy.Stream class to change the on_data function
class MyStream(tweepy.Stream):

    #this function gets called when a tweet meets the search criteria
    def on_data(self, data):
        tweet = json.loads(data.decode('utf-8'))
        tweet['_id'] = tweet['id']
        collection.insert_one(tweet)

### Starten des Streams

Um den Stream starten zu können, reicht es, ein Objekt der gerade erstellten Klasse MyStream mit den benötigten Authentifizierungstoken aus der .env-Datei zu erstellen und die Methode filter() mit den gewünschten Filterwörtern aufzurufen.  
Um ein Abbrechen des Streams zu vermeiden, wird eine zusätzliche Fehlerbehandlung mittels try-Except Statements eingebaut, die nur bei einer Unterbrechung mittels des Keyboards den Prozess abbricht und ansonsten immer neu startet.

In [ ]:
def start_stream():
    while True:
        try:
            #setup stream authentication
            stream = MyStream(os.getenv("CONSUMER_KEY"),
                        os.getenv("CONSUMER_SECRET"),
                        os.getenv("ACCESS_TOKEN"),
                        os.getenv("ACCESS_TOKEN_SECRET"))
            stream.filter(track=list(to_track), languages = ['de'])
        except KeyboardInterrupt: 
            break
        except:
            continue

start_stream()

### Tweets aus Datenbank ziehen

Die gesammelten Tweets können dann einfach als CSV-Datei aus der Datenbank heruntergeladen werden und in der tweet_processing.py Datei eingelesen, vorverarbeitet und analysiert werden.